In [ ]:
#https://docs.arize.com/phoenix/retrieval/quickstart-retrieval
#https://colab.research.google.com/github/Arize-ai/phoenix/blob/main/tutorials/qdrant_langchain_instrumentation_search_and_retrieval_tutorial.ipynb#scrollTo=sgZQUJ0LVxaQ


In [ ]:
import phoenix as px

In [4]:
# Standard library imports
#import os
#from getpass import getpass

# Third-party library imports
import nest_asyncio
#import numpy as np
import pandas as pd

# Miscellaneous imports

# Configuration and Initialization
nest_asyncio.apply()
pd.set_option("display.max_colwidth", None)

In [5]:
#In order to make your LLM application observable, it must be instrumented. That is, the code must emit traces. The instrumented data must then be sent to an Observability backend, in our case the Phoenix server.
from phoenix.trace.langchain import LangChainInstrumentor

LangChainInstrumentor().instrument()

In [ ]:
import weaviate

client = weaviate.Client("http://localhost:8081")
weaviate_client = weaviate.connect_to_local("localhost","8081")#v4
index_name="Phoenix_test"

In [ ]:
weaviate_client.collections.delete_all()
weaviate_client.collections.list_all()

In [ ]:
from weaviate.classes.config import Configure
from weaviate.classes.config import Property, DataType

weaviate_client.collections.create(
    index_name,
    #see notes above re: the docker modules that need to be enabled for text2vec* to work correctly -e ENABLE_MODULES=text2vec-ollama
    vectorizer_config=Configure.Vectorizer.text2vec_ollama( 
        model="nomic-embed-text",    
        api_endpoint="http://host.docker.internal:11434",
    ),
    # generative_config=Configure.Generative.ollama(
    #     api_endpoint = "http://host.docker.internal:11434",
    #     model="jonphi"
    # ),

    # properties=[
    #     Property(name="page_content", data_type=DataType.TEXT),
    #     Property(name="source", data_type=DataType.INT),
    # ]

)

In [11]:
from langchain.embeddings import OllamaEmbeddings

model_name = "nomic-embed-text"
embedding = OllamaEmbeddings(model=model_name)

In [15]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter


#todo - look into keeping bullets together in available loader libraries
#loader = TextLoader("/home/jonot480/Documents/Coke OCG.txt")
loader = TextLoader("/home/jonot480/Documents/paulgraham.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024,chunk_overlap=200)
docs = text_splitter.split_documents(documents)
#print(*docs, sep='\n')

In [16]:
from langchain_community.vectorstores.weaviate import Weaviate
from langchain_community.embeddings.ollama import OllamaEmbeddings

weav = Weaviate.from_documents(
    docs, 
    embedding=embedding,
    client=client,    
    index_name=index_name,
    #prefer_grpc=True, 
) 

In [18]:
from langchain.chat_models.ollama import ChatOllama
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import RetrievalQA

handler = StdOutCallbackHandler()

num_retrieved_documents = 2

retriever = weav.as_retriever(
    search_type="mmr", 
    search_kwargs={
        "k": num_retrieved_documents
    }, 
    enable_limit=True
)
chain_type = "stuff"  # stuff, refine, map_reduce, and map_rerank
chat_model_name = "gpt-4-turbo-preview"
llm = ChatOllama(model_name=chat_model_name, temperature=0.0)
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type=chain_type,
    retriever=retriever,
    metadata={"application_type": "question_answering"},
    callbacks=[handler],
)

In [19]:
query_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/context-retrieval/langchain/langchain_query_dataframe_with_user_feedbackv2.parquet"
)

In [ ]:
session = px.launch_app()

In [ ]:
for i in range(10):
    row = query_df.iloc[i]
    response = chain.invoke(row["text"])
    print(response)